In [1]:
from model import init_llm
import json

In [2]:
model, tokenizer = init_llm()

WARNING 12-15 00:14:53 config.py:1668] Casting torch.bfloat16 to torch.float16.
WARNING 12-15 00:14:59 arg_utils.py:957] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 12-15 00:14:59 config.py:1021] Chunked prefill is enabled with max_num_batched_tokens=512.
INFO 12-15 00:14:59 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='/project/pi_hongyu_umass_edu/shared_llm_checkpoints/Meta-Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='/project/pi_hongyu_umass_edu/shared_llm_checkpoints/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=131072, download_dir=No

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 12-15 00:15:06 model_runner.py:1067] Loading model weights took 14.9888 GB
INFO 12-15 00:15:06 gpu_executor.py:122] # GPU blocks: 28107, # CPU blocks: 2048
INFO 12-15 00:15:06 gpu_executor.py:126] Maximum concurrency for 131072 tokens per request: 3.43x
INFO 12-15 00:15:08 model_runner.py:1395] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 12-15 00:15:08 model_runner.py:1399] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 12-15 00:15:18 model_runner.py:1523] Graph capturing finished in 10 secs.


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  8.01it/s, est. speed input: 347.76 toks/s, output: 64.69 toks/s]

[RequestOutput(request_id=0, prompt='<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat is the capital of France?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n', prompt_token_ids=[128000, 128000, 128006, 9125, 128007, 271, 38766, 1303, 33025, 2696, 25, 6790, 220, 2366, 18, 198, 15724, 2696, 25, 220, 1627, 10263, 220, 2366, 19, 271, 128009, 128006, 882, 128007, 271, 3923, 374, 279, 6864, 315, 9822, 30, 128009, 128006, 78191, 128007, 271], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text='The capital of France is Paris.', token_ids=(791, 6864, 315, 9822, 374, 12366, 13, 128009), cumulative_logprob=None, logprobs=None, finish_reason=stop, stop_reason=None)], finished=True, metrics=RequestMetrics(arrival_time=1734221719.5451925, last_token_time=1734221719.5451925, first_schedule

In [ ]:
import json
split = 0
file_name = f'Instruction_{str(split)}_version2'
with open(f'{file_name}.json', 'r') as file:
    data = json.load(file)
from tqdm import tqdm
def prepare_prompt(instruction, context, question):
    return f"{instruction}\n\n\n###CONTEXT: {context} \n\n\n###QUESTION: {question}"

for i in tqdm(range(len(data))):
    k = prepare_prompt(data[i]["question"], data[i]["context"], data[i]["question"])
    messages = [{"role": "user", "content": k}]
    formatted_prompt =  tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    output = model.generate(formatted_prompt)
    data[i]["llama_out"] = output[0].outputs[0].text
with open(f"{file_name}_w_llama.json", "w") as outfile:
    json.dump(data, outfile)

In [2]:
system_prompt_w_context = \
        '''
        You are a highly knowledgeable medical assistant specialized in occupational health and toxicology.
        Your role is to analyze detailed medical scenarios and 
        accurately assess health risks based on the provided medical context and multiple-choice options
        '''

system_prompt_wo_context = \
    '''
    You are a highly knowledgeable medical assistant specialized in occupational health and toxicology.
    Your role is to analyze detailed medical scenarios and 
    accurately assess health risks based on the provided multiple-choice options
    '''

In [50]:
def run_vLLM(idx):
    out_file = f'responses/vLLM_out{idx}.json'
    file_name = f'vLLm_prompts/prompt_{idx}.json'
    ans = []
    ans_context = []
    original_ans = []
    full_option = []
    with open(file_name, 'r') as file:
        allprompt = json.load(file)

    for i in range(len(allprompt)):
        ans_org = allprompt[i]["orginal_ans"]
        try:
            original_ans.append(ans_org[0])
        except:
            original_ans.append("Z")
        full_option.append(str(ans_org))

    for i in range(len(allprompt)):
        # prompt = allprompt[i]["prompt"]
        context_prompt = allprompt[i]["context_prompt"]
        messages = [{"role": "user", "content": context_prompt}]
        formatted_prompt =  tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        output = model.generate(formatted_prompt)
        ans_context.append(output[0].outputs[0].text)


    for i in range(len(allprompt)):
        prompt = allprompt[i]["prompt"]
        messages = [{"role": "user", "content": prompt}]
        formatted_prompt =  tokenizer.apply_chat_template(messages,
                                                          tokenize=False, 
                                                          add_generation_prompt=True)
        output = model.generate(formatted_prompt)
        ans.append(output[0].outputs[0].text)

    da = {"chunk_set_id": [idx]*len(ans),
        "ans_wo_context": ans,
          "ans_w_context": ans_context,
          "orginal_ans": original_ans, 
          "full_option":full_option}    
    with open(out_file, "w") as outfile:
        json.dump(da, outfile)
    return da


In [51]:
all_fdf = []
for idx in range(10):
    d = pd.DataFrame(run_vLLM(idx))
    all_fdf.append(d)
    


Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 22.12it/s, est. speed input: 3874.97 toks/s, output: 44.52 toks/s]


In [52]:
pd.concat(all_fdf,axis=0).reset_index(drop=True).to_csv("responses/final_10.csv", index=False)

In [46]:
pd.concat(all_fdf,axis=0)

,ans_wo_context,ans_w_context,orginal_ans,full_option
0,B,B,B,B: Rinse the affected skin thoroughly with wat...
1,D,D,D,D: Prolonged exposure to low concentrations of...
2,D,B,B,B: 0.03 mg/m3. This is the correct answer beca...
3,A,D,A,A
4,B,1,A,A: Decreased phase I reactions due to enzyme s...
...,...,...,...,...
253,"A\n\nHowever, a more accurate answer with a sl...",B,T,The best answer is B: Assess the patient's HIV...
254,<opt>8 weeks</opt>,4,B,B: 4 weeks. The best answer is B. PEP for HIV ...
255,D,D,D,D: To decrease the probability of HIV seroconv...
256,D,C,C,C: The risk is negligible since the syringe wa...
